## Zadanie PS 9 / praca domowa #4

Na podstawie dwóch tabel zapisanych w plikach csv, utwórz nową zawierającą wszystkich klientów. Zignoruj kolumny DOB, cusfirstorder, cuszone, cusannualsales, currentbalance. Zwróć uwagę na poniższe punkty.

1. Należy ujednolicić pole, które jest kluczem głównym w tabeli Customer oraz w plikach z danymi, które mają być załadowane do tabeli Customer.

2. Kod pocztowy powinien być reprezentowany przez 5 cyfr (jeśli kod zawiera więcej cyfr, należy je pominąć)

3. Stan powinien być zawsze reprezentowany przez dwie duże litery.

4. Należy zmienić każde wystąpienie skrótu Pgh na pełną nazwę miasta tj. Pittsburgh.

5. Nazwa klienta i adres powinny być reprezentowana przez pojedyńcze pola. Jeśli nazwa klienta jest reprezentowana przez dwa pola, wówczas należy dokonać konkatenacji (np. pola cusaddress1 oraz cusaddress2 w BranchCustomers powinny być reprezentowane przez jedną kolumnę).

6. Należy zmienić nazwę klienta na wielkie litery i usunąć niepotrzebne początkowe i końcowe znaki białe (spacje)

7. Należy oczyścić z białych znaków też kolumnę nazwy miasta

8. Należy zmienić nazwę kolumny cusrecentorder na RecentOrder, dla danych z NewCustomer w tej kolumnie proszę wstawić datę 1 marca 2020 w odpowiednim formacie

9. Ostateczna wersja tabeli Customer powinna zawierać tylko jednego klienta o podanej nazwie.

Uwaga: Sprawozdanie można przesłać w postaci notatnika jupyter, pliku HTML lub pliku dokumentu pdf. Jeśli zdecydujecie się Państwo na PDF lub HTML, to należy w nim umieścić teści poszczgólnych poleceń z komentarzem do czego służą oraz zawartość końcowej tabeli Customer.


In [107]:
import pandas as pd

bc = pd.read_csv("BranchCustomers.csv")
nc = pd.read_csv("NewCustomers.csv")

In [108]:
# wyrzucenie niepotrzebnych kolumn w bc drop -> cusfirstorder, cuszone, cusannualsales, currentbalance
bc.drop(
    ["cusfirstorder", "cuszone", "cusannualsales", "currentbalance"],
    axis=1,
    inplace=True,
)

# wyrzucenie niepotrzebnych kolumn w nc drop -> DOB
nc.drop(["DOB"], axis=1, inplace=True)

_5. Nazwa klienta i adres powinny być reprezentowana przez pojedyńcze pola. Jeśli nazwa klienta jest reprezentowana przez dwa pola, wówczas należy dokonać konkatenacji (np. pola cusaddress1 oraz cusaddress2 w BranchCustomers powinny być reprezentowane przez jedną kolumnę)._

⬇️


In [109]:
# połączenie kolumn fname, middle i lname w bc
bc["Name"] = (
    bc["cusfname"]
    + " "
    + (bc["cusmiddle"] + "." + " ").fillna("")
    + bc["cuslname"].fillna("")
)

# połączenie kolumn add1 i add2 w bc
bc["Address"] = bc["cusadd1"] + " " + bc["cusadd2"].fillna("")

# usunięcie niepotrzebnych kolumn w bc
bc.drop(
    ["cusfname", "cusmiddle", "cuslname", "cusadd1", "cusadd2"], axis=1, inplace=True
)

In [111]:
# zmiana nazw nc
nc.rename(columns=lambda x: x.capitalize(), inplace=True)

# zmiana nazw bc usuwając prefix cus i duża litera
bc.rename(columns=lambda x: x.replace("cus", "").capitalize(), inplace=True)
bc.rename(columns={"Num": "Id"}, inplace=True)

In [112]:
# dodanie kolumn email i phone do nc
nc["Email"] = nc.apply(
    lambda row: row["Contact"] if row["Contactpref"] == "email" else pd.NA, axis=1
)
nc["Phone"] = nc.apply(
    lambda row: row["Contact"] if row["Contactpref"] == "phone" else pd.NA, axis=1
)

# usunięcie kolumn contact i contactpref z nc
nc.drop(["Contact", "Contactpref"], axis=1, inplace=True)

In [113]:
# połącznie dwóch data frame'ów
df = pd.concat([nc, bc])

# zamiana wszystkich NaN na NA
df.fillna(pd.NA, inplace=True)

_2. Kod pocztowy powinien być reprezentowany przez 5 cyfr (jeśli kod zawiera więcej cyfr, należy je pominąć)_

⬇️


In [114]:
df["Zip"] = df["Zip"].astype(str).str[:5]

_3. Stan powinien być zawsze reprezentowany przez dwie duże litery._

⬇️


In [115]:
df["State"] = df["State"].str.upper()

_6. Należy zmienić nazwę klienta na wielkie litery i usunąć niepotrzebne początkowe i końcowe znaki białe (spacje)_

⬇️


In [116]:
# zmiana nazw klientów na wielkie litery i usunięcie białych znaków
df["Name"] = df["Name"].str.upper().str.strip()

_4. Należy zmienić każde wystąpienie skrótu Pgh na pełną nazwę miasta tj. Pittsburgh._

_7. Należy oczyścić z białych znaków też kolumnę nazwy miasta_

⬇️


In [117]:
# zmiana nazwy kolumny Recentorder na RecentOrder
df.rename(columns={"Recentorder": "RecentOrder"}, inplace=True)

# zamiana wszystkich pustch miejsc datą 1/03/2020
df.loc[df["RecentOrder"].isnull(), "RecentOrder"] = "1/03/2020"

_8. Należy zmienić nazwę kolumny cusrecentorder na RecentOrder, dla danych z NewCustomer w tej kolumnie proszę wstawić datę 1 marca 2020 w odpowiednim formacie_

⬇️


In [118]:
# usuwanie białych znaków i podmianka igonrujące wielkość znaków
df["City"] = df["City"].str.strip().str.replace("pgh", "Pittsburgh", case=False)

In [119]:
# zamiana typu kolumny
df["RecentOrder"] = pd.to_datetime(df["RecentOrder"])

_9. Ostateczna wersja tabeli Customer powinna zawierać tylko jednego klienta o podanej nazwie._

⬇️


In [120]:
# wyrzucenie duplikatów i pozostawnie ostatniego wystąpienia
df.drop_duplicates(subset="Name", keep="last", inplace=True)

_1. Należy ujednolicić pole, które jest kluczem głównym w tabeli Customer oraz w plikach z danymi, które mają być załadowane do tabeli Customer._

⬇️


In [121]:
# reset indexów
df.reset_index(drop=True, inplace=True)

# ujednolicenie id
df["Id"] = "C" + (df.index + 1).astype(str)

In [122]:
df

,Id,Name,Address,City,State,Zip,Email,Phone,RecentOrder
0,C1,JILL FRANKS,111 Lake Dr,Pittsburgh,PA,15212,jf773@hotmail.com,<NA>,2020-01-03
1,C2,SAL YATES,57 College Way,Morgantown,WV,26501,<NA>,724-234-6542,2020-01-03
2,C3,WILL RATLIFF,544 Fifth Ave,Pittsburgh,PA,15219,ratty@mail.com,<NA>,2020-01-03
3,C4,FRAN VICKS,10332 West Street,McCandless,PA,15234,<NA>,<NA>,2020-01-03
4,C5,BILLY MURRAY,2333 N Bellefield,Pittsburgh,PA,15213,bmbm@pitt.edu,<NA>,2020-01-03
5,C6,KLINE'S,500 West South Street,Pittsburgh,PA,15219,klines2@klines.com,<NA>,2020-01-03
6,C7,JIM A. SEMP,123 Main Street Suite 23,Pittsburgh,PA,15217,jsemp@mail.com,123-654-2344,2018-05-01
7,C8,LAURA VAIL,23 Oak Lane,Morgantown,WV,26501,laura.vail@hotmail.com,756-345-6789,2016-06-01
8,C9,AL'S BAKERY,846 Fifth Ave,Pittsburgh,PA,15219,al@alsbakery.com,213-543-2342,2011-08-12
9,C10,JANET R. WENDELL,23212 Fern Circle Apt 100,Washington,DC,20001,jwen@yahoo.com,434-786-3453,2014-11-24
